# 1.1. Merge GeoTiff Maps
---
Last edited: 05/02/2022 by Jeongkyung Won<br>
This code does: 
*   loads small pieces of georeferenced 1970 NGII Maps (compressed version)
*   merge them into one large chunk covering entire South Korea

*NGII: National Geographic Information Institute, KR

In [ ]:
!pip install rioxarray 
!pip install rasterio

In [ ]:
import rioxarray as rxr
import rasterio as rio
import glob, os
import numpy as np

from tqdm import notebook, trange
from rasterio.crs import CRS
from rasterio.merge import merge

In [ ]:
import requests
import zipfile
from urllib.parse import unquote # for url string conversion
import re
from tqdm import tqdm

def download_dropbox(url, folder):
  ## version 2.0, last modified by Hyunjoo Yang (hyang@sogang.ac.kr) on Jan. 14 2022
  ## This function downloads dropbox shareable link to a local folder (tested for file downloading, but not for shared folder!)
  # url: dropbox shareable link for downloading
  # folder: where to download
  
  headers = {'user-agent': 'Wget/1.16 (linux-gnu)'}
  r = requests.get(url, stream=True, headers=headers)
  
  # convert dropbox shareable link ("dl=0" to "dl=1")
  url = url.replace("?dl=0", "?dl=1")

  # check if the url returns valid status code (200)
  if r.status_code == 200:
    print('The url is valid.')

    # grab filename from the url, using regular expressions (and replace space to "_" )
    file_name = unquote(re.search(r'\/([^\/]+\.([\w]+))\?dl=([01])$', url).group(1)).replace(" ", "_")

    folder_n_fname = os.path.join(folder, file_name)

    # download
    print('Begin downloading < {} >'.format(folder_n_fname))


    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    t = tqdm(total=total_size, unit='B', unit_scale=True)
    with open(file_name, 'wb') as f:
      for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
    t.close()

    print('Finished downloading < {} >'.format(folder_n_fname))

    # unzip if zip file
    if file_name.endswith('.zip'):
      print('Extracting zip file...')
      with zipfile.ZipFile(file_name,"r") as zip_ref:
        zip_ref.extractall(folder)
    else:
      if not os.path.exists(folder):
        os.makedirs(folder)
      shutil.move(file_name, folder_n_fname)

    print('Done!')
  else:
    raise ValueError('Nothing to download: dropbox link is not valid. Check the URL link again')

In [ ]:
## dropbox shareable url link

url = 'https://www.dropbox.com/s/mn03albcarxt6zj/compressed_version.zip?dl=0' # georeferenced 1970 NGII maps (compressed version)

# file path for download and unzip
url_path = './compressed_version'

# download dropbox shared file and unzip it
download_dropbox(url, url_path)

In [ ]:
# Folder with GeoTIFF files to merge
  folder_rasters_to_merge = "./compressed_version/*.tif"
  
# Baseline coordinate system
  CRS = 4326
  
# Output directory
  outdr = "./raster_merged/NGII_1970_EPSG_4326.tif"
  
# Creates the list of directories of GeoTIFFs
  dir_rasters_to_merge = glob.glob(folder_rasters_to_merge)
  
  rasters_to_merge = []
  
  for dir_raster in dir_rasters_to_merge :
    
    r = rio.open(dir_raster)
    
    if r.crs == CRS :
        rasters_to_merge.append(r)
    else : 
        print(dir_raster, r.crs)
        continue
        
    print(len(rasters_to_merge))
    
    rasters, output = merge(rasters_to_merge)
    
    output_meta = r.meta.copy()
    
    output_meta.update({'driver':'GTiff',
                    'height':rasters.shape[1],
                    'width':rasters.shape[2],
                    'transform':output,
                    'compress':'deflate'}) # This option compresses tiff
                    
    with rio.open(outdr, 'w', **output_meta) as r :
      r.write(rasters)